In [2]:
import os
import subprocess
from dotenv import load_dotenv
import sys
import os
import importlib
for k,v in list(sys.modules.items()):
    if k.startswith('tools') or k.startswith('.env'):
        importlib.reload(v)


# Load the environment variables from the .env file
load_dotenv()

FOLDER_ID_TO_EMAIL = {
    '1Qdrs4naVqJH2KIcr1maQ3vuq5DGuDK-G': 'scha@cancelledfoodcoupon.com',
    '1AUSninKPQ9mZXFaISKAXPRv4RzpB9oNx': 'mike@mantisnetworks.co',
    '1UHH7ZuFS8anO_NIFqe25SHqPun_stmeQ': 'clint@mantisnetworks.co',
    '1ibUUpCy74WUROr5TSa-pLYLYM6ivUUmZ': 'loren@mantisnetworks.co',
    '1SwickgZ8MDK_BIyL7IhSn0oZdVxAMzHE': 'joshua.stapleton.ai@gmail.com',
    '1BO0yHZO8CfrSzX2SvWhD1uUbq_M3L7X2': 'bartdenil12@gmail.com',
}

# folder ids from google drive
FOLDER_NAME_TO_FOLDER_ID = {
    'audios_scha': '1Qdrs4naVqJH2KIcr1maQ3vuq5DGuDK-G',
    'audios_mike': '1AUSninKPQ9mZXFaISKAXPRv4RzpB9oNx',
    'audios_clint': '1UHH7ZuFS8anO_NIFqe25SHqPun_stmeQ',
    'audios_loren': '1ibUUpCy74WUROr5TSa-pLYLYM6ivUUmZ',
    'audios_josh': '1SwickgZ8MDK_BIyL7IhSn0oZdVxAMzHE',
    'audios_bart': '1BO0yHZO8CfrSzX2SvWhD1uUbq_M3L7X2',
}


In [3]:
def process_specific_file(target_file, local_folder):
    base = os.path.splitext(target_file)[0]
    input_file = os.path.join(local_folder, target_file)
    output_file = os.path.join(local_folder, f"{base}.wav")

    if target_file.endswith('.m4a'):
        print(f"Processing {target_file} file as .m4a...")
        # Run the ffmpeg command for .m4a files
        subprocess.run(['ffmpeg', '-i', input_file, '-ar', '16000', output_file], stdout=subprocess.DEVNULL, stderr=subprocess.STDOUT)
    elif target_file.endswith('.mp3'):
        print(f"Processing {target_file} file as .mp3...")
        # Run the ffmpeg command for .mp3 files
        subprocess.run(['ffmpeg', '-i', input_file, '-ar', '16000', output_file], stdout=subprocess.DEVNULL, stderr=subprocess.STDOUT)
    else:
        print(f"Unsupported file format: {target_file}")
        return  # Exit the function

    # Common processing for all supported file formats
    subprocess.run(['./main', '-f', output_file], stdout=subprocess.DEVNULL, stderr=subprocess.STDOUT)


In [6]:
import os
from tools.gpt_functions import generate_gpt4_response
from tools.notification_functions import send_email


def fred(filename, folder, local_folder):
    filename = local_folder +'/'+filename.split('.')[0]+".wav.txt"
    with open(filename, 'r') as file:
        transcript = file.read()

    # prompt = "Summarize, generate actionables, and follow instructions for the following transcript:" + transcript
    prompt = transcript
    response = generate_gpt4_response(prompt)

    receiving_email_address = FOLDER_ID_TO_EMAIL.get(folder)  # Look up the email address

    # if receiving_email_address:  # If an email address was found
    #     send_email(os.environ.get('SENDING_EMAIL_ADDRESS'), receiving_email_address, "FRED response for " + filename, response + "\n\n-----------------------\n\nTRANSCRIPT:\n" + transcript, os.environ.get('EMAIL_PASSWORD'))

    send_email(os.environ.get('SENDING_EMAIL_ADDRESS'), 'joshua.stapleton.ai@gmail.com', "FRED response for " + filename, response + "\n\n-----------------------\n\nTRANSCRIPT:\n" + transcript, os.environ.get('EMAIL_PASSWORD'))

In [7]:
from __future__ import print_function
import os.path
import time
import io
from googleapiclient.discovery import build
from googleapiclient.http import MediaIoBaseDownload
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request
import pickle
from googleapiclient.errors import HttpError


# If modifying these SCOPES, delete the file token.pickle.
SCOPES = ['https://www.googleapis.com/auth/drive.readonly']

def main():
    try:
        """Shows basic usage of the Drive v3 API.
        Lists the names and ids of the first 10 files the user has access to.
        """
        creds = None
        # The file token.pickle stores the user's access and refresh tokens, and is
        # created automatically when the authorization flow completes for the first
        # time.
        if os.path.exists('token.pickle'):
            with open('token.pickle', 'rb') as token:
                creds = pickle.load(token)
        # If there are no (valid) credentials available, let the user log in.
        if not creds or not creds.valid:
            if creds and creds.expired and creds.refresh_token:
                creds.refresh(Request())
            else:
                flow = InstalledAppFlow.from_client_secrets_file(
                    'client_secret_782650429580-k51cnfcs0gmn6kdkn7t5elbchinpspo1.apps.googleusercontent.com.json', SCOPES)
                creds = flow.run_local_server(port=0)
            # Save the credentials for the next run
            with open('token.pickle', 'wb') as token:
                pickle.dump(creds, token)

        drive_service = build('drive', 'v3', credentials=creds)

        # Continuously poll Google Drive folder for new files
        downloaded_files = set()
        for local_folder in FOLDER_NAME_TO_FOLDER_ID.keys():
            downloaded_files.update(os.listdir(local_folder))
        print("Downloaded files:", downloaded_files)
        # Continuously poll Google Drive folder for new files
        while True:
            for local_folder, folder_id in FOLDER_NAME_TO_FOLDER_ID.items():
                print(local_folder)
                request = drive_service.files().list(
                q="'{}' in parents and trashed = false".format(folder_id),
                fields='nextPageToken, files(id, name)',
                pageToken=None).execute()
                
                # Get all files in the Google Drive folder
                all_files = request.get('files', [])
                print("All files currently in GD:", all_files)
                # Remove already downloaded files
                files_to_download = [file for file in all_files if file['name'] not in downloaded_files]
                print("Files to download:", files_to_download)
                
                # only for files which have not been downloaded
                for file in files_to_download:
                    # Download file
                    request = drive_service.files().get_media(fileId=file['id'])
                    fh = io.BytesIO()
                    downloader = MediaIoBaseDownload(fh, request)
                    done = False
                    while done is False:
                        status, done = downloader.next_chunk()
                    with io.open(os.path.join(local_folder, file['name']), 'wb') as f:
                        print("Writing file...", file['name'])
                        fh.seek(0)
                        f.write(fh.read())
                        
                    # Add file to the record of downloaded files
                    downloaded_files.add(file['name'])
                    # process the file to .wav
                    process_specific_file(file['name'], local_folder)
                    fred(filename=file['name'], folder=folder_id, local_folder=local_folder)
                time.sleep(5)


    except HttpError as error:
        print(f"An HTTP error occurred: {error}")

if __name__ == '__main__':
    main()
# await main() # for notebook since event loop already created

Downloaded files: {'Recording_334.wav', 'Email octopi 18072023.wav.txt', 'Voice 004.wav', 'Recording_327.wav.txt', 'Recording_325.wav.txt', 'Recording_331.wav.txt', 'Voice 003.m4a', 'Voice 004 - Copy.wav', 'Mantis support 26072023.wav.txt', 'Voice 002.m4a', 'Recording_327.m4a', 'Recording_325.m4a', 'Email octopi 18072023.m4a', 'Email octopi 18072023.wav', 'Recording_330.m4a', 'Recording_327.wav', 'Recording_323.wav', 'Octopi trust questions 26072023.wav', 'Recording_323.m4a', 'Recording_329.m4a', 'Voice 002.wav', 'Mantis support 26072023.wav', 'Wilkes Road.wav', 'Audio from ☕', 'Recording_331.wav', 'Recording_325.wav', 'Voice 004 - Copy.m4a', 'Voice 004.m4a', 'Recording_329.wav.txt', 'Mantis support 26072023.m4a', 'Octopi trust questions 26072023.m4a', 'Recording_334.wav.txt', 'Recording_330.wav.txt', 'Recording_332.wav.txt', 'Voice 002.wav.txt', 'Octopi trust questions 26072023.wav.txt', 'Voice 004 - Copy.wav.txt', 'Voice 004.wav.txt', 'Recording_329.wav', 'Wilkes Road.wav.txt', 'Reco

FileNotFoundError: [Errno 2] No such file or directory: 'audios_bart/Audio from Bi Africa.wav.txt'

In [ ]:
# Old working but inefficient version

# from __future__ import print_function
# import os.path
# import time
# import io
# from googleapiclient.discovery import build
# from googleapiclient.http import MediaIoBaseDownload
# from google_auth_oauthlib.flow import InstalledAppFlow
# from google.auth.transport.requests import Request
# import pickle
# from googleapiclient.errors import HttpError


# # If modifying these SCOPES, delete the file token.pickle.
# SCOPES = ['https://www.googleapis.com/auth/drive.readonly']

# def main():
#     try:
#         """Shows basic usage of the Drive v3 API.
#         Lists the names and ids of the first 10 files the user has access to.
#         """
#         creds = None
#         # The file token.pickle stores the user's access and refresh tokens, and is
#         # created automatically when the authorization flow completes for the first
#         # time.
#         if os.path.exists('token.pickle'):
#             with open('token.pickle', 'rb') as token:
#                 creds = pickle.load(token)
#         # If there are no (valid) credentials available, let the user log in.
#         if not creds or not creds.valid:
#             if creds and creds.expired and creds.refresh_token:
#                 creds.refresh(Request())
#             else:
#                 flow = InstalledAppFlow.from_client_secrets_file(
#                     'client_secret_782650429580-k51cnfcs0gmn6kdkn7t5elbchinpspo1.apps.googleusercontent.com.json', SCOPES)
#                 creds = flow.run_local_server(port=0)
#             # Save the credentials for the next run
#             with open('token.pickle', 'wb') as token:
#                 pickle.dump(creds, token)

#         drive_service = build('drive', 'v3', credentials=creds)

#         # Continuously poll Google Drive folder for new files
#         downloaded_files = set()
#         for local_folder in FOLDER_NAME_TO_FOLDER_ID.keys():
#             downloaded_files.update(os.listdir(local_folder))
#         print(downloaded_files)

#         # Continuously poll Google Drive folder for new files
#         while True:
#             for local_folder, folder_id in FOLDER_NAME_TO_FOLDER_ID.items():
#                 request = drive_service.files().list(
#                 q="'{}' in parents and trashed = false".format(folder_id),
#                 fields='nextPageToken, files(id, name)',
#                 pageToken=None).execute()
                
#                 for file in request.get('files', []):
#                     # Skip download if file has already been downloaded
#                     if file['name'] in downloaded_files:
#                         print("File", file['name'], "has already been downloaded.")
#                         hoi = 1

#                     # Download file
#                     else:
#                         request = drive_service.files().get_media(fileId=file['id'])
#                         fh = io.BytesIO()
#                         downloader = MediaIoBaseDownload(fh, request)
#                         done = False
#                         while done is False:
#                             status, done = downloader.next_chunk()
#                         with io.open(os.path.join(local_folder, file['name']), 'wb') as f:
#                             print("Writing file...", file['name'])
#                             fh.seek(0)
#                             f.write(fh.read())
                            
#                         # Add file to the record of downloaded files
#                         downloaded_files.add(file['name'])
#                         # process the file to .wav
#                         process_specific_file(file['name'], local_folder)
#                         fred(filename=file['name'], folder=folder_id, local_folder=local_folder)
#                 time.sleep(5)

#     except HttpError as error:
#         print(f"An HTTP error occurred: {error}")

# if __name__ == '__main__':
#     main()